#### connect to bigquery

In [1]:
from google.cloud import bigquery

client = bigquery.Client(project="sales-pulse-etl-dashboard")
project_id = 'sales-pulse-etl-dashboard'
dataset_id = 'sales_data'
table_id = 'advertising_metrics'
table_ref = f"{project_id}.{dataset_id}.{table_id}"

In [2]:
print(f"Table reference: {table_ref}")

Table reference: sales-pulse-etl-dashboard.sales_data.advertising_metrics


In [3]:
%load_ext bigquery_magics

##### sql queries

In [4]:
%%bigquery result --project {project_id}
SELECT *
FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
display(result)

,TV,Radio,Newspaper,Sales,Total_Spend,TV_Percentage,Radio_Percentage,Newspaper_Percentage,TV_Radio_Interaction,TV_Newspaper_Interaction,Log_Sales,Log_TV,ROAS
0,0.7,39.6,8.7,1.6,49.0,0.014286,0.808163,0.177551,27.72,6.09,0.955511,0.530628,0.032653
1,4.1,11.6,5.7,3.2,21.4,0.191589,0.542056,0.266355,47.56,23.37,1.435085,1.629241,0.149533
2,8.6,2.1,1.0,4.8,11.7,0.735043,0.179487,0.085470,18.06,8.60,1.757858,2.261763,0.410256
3,5.4,29.9,9.4,5.3,44.7,0.120805,0.668904,0.210291,161.46,50.76,1.840550,1.856298,0.118568
4,13.1,0.4,25.6,5.3,39.1,0.335038,0.010230,0.654731,5.24,335.36,1.840550,2.646175,0.135550
5,7.3,28.1,41.4,5.5,76.8,0.095052,0.365885,0.539062,205.13,302.22,1.871802,2.116256,0.071615
6,13.2,15.9,49.6,5.6,78.7,0.167726,0.202033,0.630241,209.88,654.72,1.887070,2.653242,0.071156
7,8.4,27.2,2.1,5.7,37.7,0.222812,0.721485,0.055703,228.48,17.64,1.902108,2.240710,0.151194
8,17.2,4.1,31.6,5.9,52.9,0.325142,0.077505,0.597353,70.52,543.52,1.931521,2.901422,0.111531
9,19.4,16.0,22.3,6.6,57.7,0.336222,0.277296,0.386482,310.40,432.62,2.028148,3.015535,0.114385


average sales

In [6]:
%%bigquery result --project {project_id}
SELECT AVG(Sales) AS avg_sales, AVG(Total_Spend) AS avg_total_spend, AVG(TV) AS avg_tv, AVG(Radio) AS avg_radio, AVG(Newspaper) AS avg_newspaper
FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`;

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
display(result)

,avg_sales,avg_total_spend,avg_tv,avg_radio,avg_newspaper
0,14.0225,200.8605,147.0425,23.264,30.554


ROAS levels

In [8]:
%%bigquery result --project sales-pulse-etl-dashboard
SELECT 
    MIN(ROAS) AS min_roas,
    MAX(ROAS) AS max_roas,
    AVG(ROAS) AS avg_roas,
    APPROX_QUANTILES(ROAS, 100)[OFFSET(25)] AS Q1,
    APPROX_QUANTILES(ROAS, 100)[OFFSET(50)] AS median_roas,
    APPROX_QUANTILES(ROAS, 100)[OFFSET(75)] AS Q3
FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`;

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
display(result)

,min_roas,max_roas,avg_roas,Q1,median_roas,Q3
0,0.032653,0.410256,0.078731,0.063013,0.073449,0.088212


In [10]:
%%bigquery result --project sales-pulse-etl-dashboard
SELECT *
FROM (
    SELECT 
        Sales,
        Total_Spend,
        TV,
        Radio,
        Newspaper,
        ROAS,
        CASE 
            WHEN ROAS >= 0.088 THEN 'High Efficiency'
            WHEN ROAS >= 0.063 THEN 'Medium Efficiency'
            ELSE 'Low Efficiency'
        END AS efficiency_category
    FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`
    ORDER BY RAND()
    LIMIT 20
)
ORDER BY ROAS DESC;

Query is running:   0%|          |

Downloading:   0%|          |

In [11]:
display(result)

,Sales,Total_Spend,TV,Radio,Newspaper,ROAS,efficiency_category
0,10.8,86.4,39.5,41.1,5.8,0.125000,High Efficiency
1,6.7,54.2,18.7,12.1,23.4,0.123616,High Efficiency
2,5.9,52.9,17.2,4.1,31.6,0.111531,High Efficiency
3,8.8,79.1,26.8,33.0,19.3,0.111252,High Efficiency
4,10.1,104.9,43.1,26.7,35.1,0.096282,High Efficiency
5,9.7,107.2,94.2,4.9,8.1,0.090485,High Efficiency
6,11.5,132.3,107.4,14.0,10.9,0.086924,Medium Efficiency
7,9.7,112.3,97.5,7.6,7.2,0.086376,Medium Efficiency
8,10.6,124.9,87.2,11.8,25.9,0.084868,Medium Efficiency
9,15.0,184.8,142.9,29.3,12.6,0.081169,Medium Efficiency


In [12]:
%%bigquery result --project {project_id}
WITH enriched AS (
  SELECT 
    Sales,
    Total_Spend,
    ROAS,
    TV,
    Radio,
    Newspaper,
    TV_Percentage,
    Radio_Percentage,
    Newspaper_Percentage,
    TV_Radio_Interaction,
    TV_Newspaper_Interaction,
    Log_Sales,
    Log_TV,
    CASE 
      WHEN ROAS < 0.063 THEN 'Low Efficiency'
      WHEN ROAS BETWEEN 0.063 AND 0.088 THEN 'Medium Efficiency'
      ELSE 'High Efficiency'
    END AS efficiency_category
  FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`
)
SELECT 
  efficiency_category,
  COUNT(*) AS num_records,
  AVG(Sales) AS avg_sales,
  AVG(Total_Spend) AS avg_total_spend,
  AVG(ROAS) AS avg_roas,
  AVG(TV_Percentage) AS avg_tv_percentage,
  AVG(Radio_Percentage) AS avg_radio_percentage,
  AVG(Newspaper_Percentage) AS avg_newspaper_percentage,
  AVG(TV_Radio_Interaction) AS avg_tv_radio_interaction,
  AVG(TV_Newspaper_Interaction) AS avg_tv_newspaper_interaction,
  AVG(Log_Sales) AS avg_log_sales,
  AVG(Log_TV) AS avg_log_tv
FROM enriched
GROUP BY efficiency_category
ORDER BY avg_roas DESC;


Query is running:   0%|          |

Downloading:   0%|          |

In [13]:
display(result)

,efficiency_category,num_records,avg_sales,avg_total_spend,avg_roas,avg_tv_percentage,avg_radio_percentage,avg_newspaper_percentage,avg_tv_radio_interaction,avg_tv_newspaper_interaction,avg_log_sales,avg_log_tv
0,High Efficiency,51,10.296078,100.027451,0.114468,0.564420,0.229528,0.206052,1590.522941,884.814510,2.365006,3.842281
1,Medium Efficiency,100,15.996000,221.020000,0.073424,0.686643,0.133959,0.179398,4791.650800,5074.714800,2.782139,4.887336
2,Low Efficiency,49,13.873469,264.667347,0.052367,0.778965,0.071996,0.149038,2811.841429,7490.372653,2.655461,5.181709


In [14]:
%%bigquery result --project {project_id}
SELECT 
    Sales,
    Total_Spend,
    ROAS,
    RANK() OVER (ORDER BY Sales DESC) AS sales_rank
FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`
ORDER BY sales_rank
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
display(result)

,Sales,Total_Spend,ROAS,sales_rank
0,27.0,367.6,0.073449,1
1,26.2,402.4,0.065109,2
2,25.5,391.8,0.065084,3
3,25.4,336.5,0.075483,4
4,25.4,315.7,0.080456,4
5,25.4,383.2,0.066284,4
6,24.7,272.5,0.090642,7
7,24.4,376.8,0.064756,8
8,24.2,358.7,0.067466,9
9,23.8,298.1,0.079839,10


In [16]:
%%bigquery result --project sales-pulse-etl-dashboard
WITH spend_buckets AS (
  SELECT
    TV,
    Radio,
    Newspaper,
    Sales,
    Total_Spend,
    ROAS,
    NTILE(4) OVER (ORDER BY Total_Spend) AS spend_quartile
  FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`
),
quartile_stats AS (
  SELECT
    spend_quartile,
    AVG(ROAS) AS avg_roas,
    AVG(Sales) AS avg_sales
  FROM spend_buckets
  GROUP BY spend_quartile
),
detailed AS (
  SELECT 
    sb.*,
    qs.avg_roas,
    qs.avg_sales,
    sb.ROAS - qs.avg_roas AS roas_diff,
    sb.Sales - qs.avg_sales AS sales_diff
  FROM spend_buckets sb
  JOIN quartile_stats qs ON sb.spend_quartile = qs.spend_quartile
)
SELECT *
FROM detailed
ORDER BY roas_diff DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

In [17]:
display(result)

,TV,Radio,Newspaper,Sales,Total_Spend,ROAS,spend_quartile,avg_roas,avg_sales,roas_diff,sales_diff
0,8.6,2.1,1.0,4.8,11.7,0.410256,1,0.111223,8.314,0.299033,-3.514
1,31.5,24.6,2.2,9.5,58.3,0.162950,1,0.111223,8.314,0.051727,1.186
2,8.4,27.2,2.1,5.7,37.7,0.151194,1,0.111223,8.314,0.039971,-2.614
3,4.1,11.6,5.7,3.2,21.4,0.149533,1,0.111223,8.314,0.038310,-5.114
4,27.5,1.6,20.7,6.9,49.8,0.138554,1,0.111223,8.314,0.027331,-1.414
5,166.8,42.0,3.6,19.6,212.4,0.092279,3,0.065121,15.768,0.027157,3.832
6,220.3,49.0,3.2,24.7,272.5,0.090642,3,0.065121,15.768,0.025521,8.932
7,38.2,3.7,13.8,7.6,55.7,0.136445,1,0.111223,8.314,0.025222,-0.714
8,184.9,43.9,1.7,20.7,230.5,0.089805,3,0.065121,15.768,0.024684,4.932
9,13.1,0.4,25.6,5.3,39.1,0.135550,1,0.111223,8.314,0.024327,-3.014


In [18]:
%%bigquery result --project sales-pulse-etl-dashboard
WITH randomized AS (
  SELECT
    *,
    IF(MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(TV AS STRING), CAST(Radio AS STRING), CAST(Newspaper AS STRING)))), 2) = 0, 'A', 'B') AS ab_group
  FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`
),
group_stats AS (
  SELECT
    ab_group,
    COUNT(*) AS n,
    AVG(Sales) AS avg_sales,
    STDDEV(Sales) AS std_sales
  FROM randomized
  GROUP BY ab_group
),
combined AS (
  SELECT
    (SELECT avg_sales FROM group_stats WHERE ab_group = 'A') AS avg_sales_A,
    (SELECT n FROM group_stats WHERE ab_group = 'A') AS n_A,
    (SELECT std_sales FROM group_stats WHERE ab_group = 'A') AS std_sales_A,
    (SELECT avg_sales FROM group_stats WHERE ab_group = 'B') AS avg_sales_B,
    (SELECT n FROM group_stats WHERE ab_group = 'B') AS n_B,
    (SELECT std_sales FROM group_stats WHERE ab_group = 'B') AS std_sales_B
)
SELECT
  avg_sales_A,
  n_A,
  std_sales_A,
  avg_sales_B,
  n_B,
  std_sales_B,
  avg_sales_A - avg_sales_B AS diff_avg_sales,
  (avg_sales_A - avg_sales_B) / SQRT(POWER(std_sales_A, 2)/n_A + POWER(std_sales_B, 2)/n_B) AS z_score,
FROM combined;


Query is running:   0%|          |

Downloading:   0%|          |

In [19]:
display(result)

,avg_sales_A,n_A,std_sales_A,avg_sales_B,n_B,std_sales_B,diff_avg_sales,z_score
0,14.121101,109,5.287408,13.904396,91,5.159132,0.216705,0.292478
